In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gravity3d import compute_gravity


Lx, Ly = 15000, 15000  
nx, ny, nz = 60, 60, 50

max_depth1 = 1600 
max_depth2 = 1100 

x_edges = np.linspace(0, Lx, nx + 1)
y_edges = np.linspace(0, Ly, ny + 1)
z_edges = np.linspace(0, max_depth1, nz + 1)

dx = x_edges[1] - x_edges[0]
dy = y_edges[1] - y_edges[0]
dz = z_edges[1] - z_edges[0]

xc = 0.5 * (x_edges[:-1] + x_edges[1:])
yc = 0.5 * (y_edges[:-1] + y_edges[1:])
zc = 0.5 * (z_edges[:-1] + z_edges[1:])

Xc2d, Yc2d = np.meshgrid(xc, yc, indexing='ij')


x1, y1 = Lx * 0.3, Ly / 2
R1 = np.sqrt((Xc2d - x1)**2 + (Yc2d - y1)**2)
Rmax1 = 3900
basin1 = max_depth1 * (1 - (R1 / Rmax1)**2)
basin1[R1 > Rmax1] = 0

x2, y2 = Lx * 0.65, Ly / 2
R2 = np.sqrt((Xc2d - x2)**2 + (Yc2d - y2)**2)
Rmax2 = 4500
basin2 = max_depth2 * (1 - (R2 / Rmax2)**2)
basin2[R2 > Rmax2] = 0

basin_depth = np.maximum(basin1, basin2)

rho_model = np.zeros((nx, ny, nz))
density_contrast = -500 

for k in range(nz):
    mask = zc[k] < basin_depth
    rho_model[:, :, k][mask] = density_contrast

Xobs, Yobs = np.meshgrid(xc, yc, indexing='ij')
Zobs = np.zeros_like(Xobs) 


gz = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, rho_model)

plt.figure(figsize=(8, 6))
plt.contourf(xc / 1e3, yc / 1e3, gz.T, 30, cmap='jet')
plt.colorbar(label='gz (mGal)')
plt.title('Gravity Anomaly from Twin Basins')
plt.xlabel('X (km)')
plt.ylabel('Y (km)')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import differential_evolution
from gravity3d import compute_gravity

def interpolate_b_spline(control_depths, control_x, control_y, full_x, full_y):
    spline = RectBivariateSpline(control_x, control_y, control_depths, kx=3, ky=3)
    return spline(full_x[:, 0], full_y[0, :])

def forward_gravity_from_controls(params, ctrl_shape, control_x, control_y,
                                  full_x, full_y, zc, x_edges, y_edges, z_edges, density_contrast):
    control_depths = params.reshape(ctrl_shape)
    basin_depth = interpolate_b_spline(control_depths, control_x, control_y, full_x, full_y)

    nx, ny = full_x.shape
    nz = len(zc)
    rho_model = np.zeros((nx, ny, nz))
    for k in range(nz):
        mask = zc[k] < basin_depth
        rho_model[:, :, k][mask] = density_contrast

    gz_pred = compute_gravity(full_x, full_y, np.zeros_like(full_x), x_edges, y_edges, z_edges, rho_model)
    return gz_pred


def objective_function(params, ctrl_shape, control_x, control_y, full_x, full_y,
                       zc, x_edges, y_edges, z_edges, density_contrast, gz_obs):
    gz_pred = forward_gravity_from_controls(params, ctrl_shape, control_x, control_y,
                                            full_x, full_y, zc, x_edges, y_edges, z_edges,
                                            density_contrast)
    return np.sqrt(np.mean((gz_pred - gz_obs)**2))


n_cx, n_cy = 8, 8
control_x = np.linspace(0, Lx, n_cx)
control_y = np.linspace(0, Ly, n_cy)
ctrl_shape = (n_cx, n_cy)

bounds = [(0, 2500)] * (n_cx * n_cy)

result = differential_evolution(
    objective_function,
    bounds=bounds,
    args=(ctrl_shape, control_x, control_y, Xc2d, Yc2d, zc,
          x_edges, y_edges, z_edges, density_contrast, gz),
    strategy='best1bin',
    popsize=10,
    maxiter=50,
    tol=1e-5,
    polish=True,
    disp=True
)

print(f"\n Optimization finished. Final RMSE: {result.fun:.4f} mGal")


control_depths_best = result.x.reshape(ctrl_shape)
basin_inverted = interpolate_b_spline(control_depths_best, control_x, control_y, Xc2d, Yc2d)